In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip3 install tensorflow==2.0 --quiet

In [42]:
import tensorflow as tf
tf.__version__

'2.0.0'

## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
import numpy as np


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prices.csv')

### Check all columns in the dataset

In [46]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
#done below

In [48]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
X=data.iloc[0:1000,0:6]

In [0]:
y=(data.iloc[0:1000,6:])/1000000

In [51]:
y.shape


(1000, 1)

In [0]:
X=X.drop(['date','symbol' ],axis=1)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer


In [0]:
transformer=Normalizer()

In [0]:
X_train=transformer.fit_transform(X_train)

In [0]:
X_test=transformer.transform(X_test)

In [59]:
X_train.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [64]:
w

<tf.Tensor: id=8435, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [65]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 248.48997
Current Loss on iteration 1 246.50015
Current Loss on iteration 2 244.66624
Current Loss on iteration 3 242.97617
Current Loss on iteration 4 241.41856
Current Loss on iteration 5 239.98308
Current Loss on iteration 6 238.66016
Current Loss on iteration 7 237.44092
Current Loss on iteration 8 236.31723
Current Loss on iteration 9 235.2817
Current Loss on iteration 10 234.32735
Current Loss on iteration 11 233.44777
Current Loss on iteration 12 232.6372
Current Loss on iteration 13 231.8901
Current Loss on iteration 14 231.20163
Current Loss on iteration 15 230.56712
Current Loss on iteration 16 229.98235
Current Loss on iteration 17 229.44344
Current Loss on iteration 18 228.94675
Current Loss on iteration 19 228.489
Current Loss on iteration 20 228.06712
Current Loss on iteration 21 227.67839
Current Loss on iteration 22 227.32007
Current Loss on iteration 23 226.98982
Current Loss on iteration 24 226.68547
Current Loss on iteration 25 226.40503
C

### Get the shapes and values of W and b

In [66]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.2638032]
 [1.3126259]
 [1.2555617]
 [1.326284 ]]
Bias:
 [2.5799315]


### Model Prediction on 1st Examples in Test Dataset

In [67]:
prediction(X_test[0:1],w,b)

<tf.Tensor: id=12541, shape=(1, 1), dtype=float32, numpy=array([[5.1586676]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Iris_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
Idata=pd.get_dummies(Iris_data)

In [70]:
Idata.sample(10)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
15,16,5.7,4.4,1.5,0.4,1,0,0
71,72,6.1,2.8,4.0,1.3,0,1,0
134,135,6.1,2.6,5.6,1.4,0,0,1
84,85,5.4,3.0,4.5,1.5,0,1,0
67,68,5.8,2.7,4.1,1.0,0,1,0
50,51,7.0,3.2,4.7,1.4,0,1,0
39,40,5.1,3.4,1.5,0.2,1,0,0
33,34,5.5,4.2,1.4,0.2,1,0,0
32,33,5.2,4.1,1.5,0.1,1,0,0
91,92,6.1,3.0,4.6,1.4,0,1,0


### Splitting the data into feature set and target set

In [0]:
X=Idata.iloc[0:,1:5]

In [0]:
y=Idata.iloc[0:,5:]

In [73]:
X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()


model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [76]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 2ms/sample - loss: 1.2169 - accuracy: 0.1905 - val_loss: 1.9373 - val_accuracy: 0.4222
Epoch 2/100
105/105 [==============================] - 0s 116us/sample - loss: 1.2110 - accuracy: 0.2095 - val_loss: 1.8370 - val_accuracy: 0.4222
Epoch 3/100
105/105 [==============================] - 0s 127us/sample - loss: 1.2022 - accuracy: 0.2095 - val_loss: 1.7465 - val_accuracy: 0.4222
Epoch 4/100
105/105 [==============================] - 0s 128us/sample - loss: 1.1919 - accuracy: 0.2095 - val_loss: 1.6697 - val_accuracy: 0.4222
Epoch 5/100
105/105 [==============================] - 0s 112us/sample - loss: 1.1874 - accuracy: 0.2095 - val_loss: 1.6043 - val_accuracy: 0.4222
Epoch 6/100
105/105 [==============================] - 0s 140us/sample - loss: 1.1737 - accuracy: 0.2190 - val_loss: 1.5493 - val_accuracy: 0.4222
Epoch 7/100
105/105 [==============================] - 0s 114us/sample - lo

In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 5         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1)                 4         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6         
Total params: 15
Trainable params: 13
Non-trainable params: 2
_________________________________________________________________


### Model Prediction

In [80]:
pred = model.predict(X_test)
pred

array([[0.11736015, 0.23633341, 0.64630646],
       [0.2929792 , 0.34626526, 0.36075553],
       [0.01265311, 0.06389449, 0.9234524 ],
       [0.15499543, 0.2710268 , 0.57397777],
       [0.06308682, 0.16849461, 0.76841855],
       [0.39031547, 0.36356914, 0.24611531],
       [0.2680042 , 0.33751166, 0.3944841 ],
       [0.07117398, 0.18040909, 0.74841684],
       [0.15146768, 0.26810074, 0.5804315 ],
       [0.21144788, 0.31026524, 0.47828686],
       [0.08868478, 0.20376366, 0.70755154],
       [0.49465197, 0.35522014, 0.15012792],
       [0.36844337, 0.36188698, 0.2696697 ],
       [0.46499404, 0.36028025, 0.17472564],
       [0.41738242, 0.36396855, 0.21864903],
       [0.10489453, 0.22293076, 0.67217463],
       [0.07259065, 0.18241659, 0.74499273],
       [0.24274178, 0.32667536, 0.43058288],
       [0.18518645, 0.29368532, 0.5211283 ],
       [0.09244476, 0.20840105, 0.6991542 ],
       [0.49449423, 0.35525253, 0.15025316],
       [0.1334612 , 0.2521563 , 0.6143825 ],
       [0.

In [85]:
for i in pred:
  predicted_num = np.argmax(i)
  print(predicted_num)

2
2
2
2
2
0
2
2
2
2
2
0
0
0
0
2
2
2
2
2
0
2
0
2
2
2
2
2
0
0
0
2
2
0
0
2
2
0
0
0
2
2
2
0
1


### Save the Model

In [0]:
model.save('Arpit.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?